In [1]:
import sgml, sgutil, sgpp, dproc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from proc_v1 import p
from ml_v1 import X_cat, X_num, X_all, target, kf, ss, config
from ml_v1 import xgb_adapter, lgb_adapter, cb_adapter, lr_adapter

In [3]:
sc = sgutil.SGCache('img', 'result', 'model')
df_train = p.fit_transform('data/train.csv').assign(
    Calories_Log = lambda x: np.log(x['Calories'] + 1),
    Duration = lambda x: np.log(x['Duration'].astype('float32')) + 1
)
df_test = p.transform('data/test.csv')
len(df_train)

750000

# 1-1 Duration을 Log로 하고 다른 변수들과 조합해봅니다.

In [4]:
from itertools import combinations, product

X_r = [i + '_r' for i in X_num]
X_o2 = [(i + '_2' if i == j else i + '_mul_'+ j, (i, j)) for i, j in product(X_num, X_num) if i <= j]
X_1d1 = [(i + '_div_' + j, (i, j)) for i, j in product(X_num, X_num) if i != j]
X_o2d1 = [
    (i + '_div_' + j, (i, j)) for (i, a), j in product(X_o2, X_num) if j not in a 
]
X_o1d2 = [
    (i + '_div_' + j, (i, j)) for i, (j, a) in product(X_num, X_o2) if i not in a 
]

X_o3 = [
    (
        i + '_3' if i == j and j == k else i + '_2_mul_' + k if i == j else i + '_2_mul_' + j if i == k  else j + '_2_mul_' + i if j == k else i + '_mul_' + j + '_mul_' + k,
        (i, j, k)
    )
    for i, j, k in product(X_num, X_num, X_num) if i <= j and i <= k and j <= k
]

X_log = [i + '_log' for i in X_num if i != 'Duration']
X_sqrt_d = [i + '_sqrt_d' for i in X_num]
X_sqrt = [
    (i + '_sqrt' if i == j else i + '_div_' + j + '_sqrt', (i, j + '_sqrt_d')) for i, j in product(X_num, X_num)
]

In [5]:
var_list = [(1 / df_train[a]).rename(a + '_r') for a in X_num]
var_list.extend([np.log(df_train[a].astype('float32')).rename(a + '_log') for a in X_num if a != 'Duration'])
var_list.extend([1 / np.sqrt(df_train[a]).rename(a + '_sqrt_d') for a in X_num])
var_list.extend([(df_train[a].astype('float32') * df_train[b].astype('float32')).rename(i) for i, (a, b) in X_o2])
var_list.extend([(df_train[a].astype('float32') / df_train[b].astype('float32')).rename(i) for i, (a, b) in X_1d1])
var_list.extend([(df_train[a].astype('float32') * df_train[b].astype('float32') * df_train[c].astype('float32')).rename(i) for i, (a, b, c) in X_o3])
df_sel = dproc.join_and_assign(
    df_train,
    pd.concat(var_list, axis = 1)
)
df_sel = dproc.join_and_assign(
    df_sel,
    pd.concat((
        [(df_sel[a] / df_sel[b]).rename(i) for i, (a, b) in X_o2d1 + X_o1d2] +
        [(df_sel[a] * df_sel[b]).rename(i) for i, (a, b) in X_sqrt]
    ), axis = 1)
).assign(
    const = 1
)
df_sel.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,Calories_Log,Age_r,...,Heart_Rate_div_Duration_sqrt,Heart_Rate_sqrt,Heart_Rate_div_Body_Temp_sqrt,Body_Temp_div_Age_sqrt,Body_Temp_div_Height_sqrt,Body_Temp_div_Weight_sqrt,Body_Temp_div_Duration_sqrt,Body_Temp_div_Heart_Rate_sqrt,Body_Temp_sqrt,const
id,,,,,,,,,,,,,,,,,,,,,
0,True,36,189.0,82.0,4.258097,101.0,41.000000,150.0,5.017280,0.027778,...,48.945595,10.049876,15.773550,6.831665,2.982311,4.527692,19.869003,4.079653,6.403124,1
1,False,64,163.0,60.0,3.079442,85.0,39.700001,34.0,3.555348,0.015625,...,48.437637,9.219544,13.490364,4.962500,3.109544,5.125248,22.623226,4.306070,6.300794,1
2,False,51,161.0,64.0,2.945910,84.0,39.799999,29.0,3.401197,0.019608,...,48.940628,9.165151,13.314895,5.572583,3.136679,4.975000,23.188536,4.342535,6.308724,1
3,True,20,192.0,90.0,4.218876,105.0,40.700001,140.0,4.948760,0.050000,...,51.120014,10.246951,16.458569,9.101855,2.937269,4.290157,19.815090,3.971913,6.379655,1
4,False,38,166.0,61.0,4.218876,102.0,40.599998,146.0,4.990433,0.026316,...,49.659443,10.099505,16.008003,6.586596,3.151171,5.198297,19.766405,4.019999,6.371813,1


In [6]:
X_sel = [i for i in df_sel.columns if i not in ('Calories', 'Calories_Log', 'const')]

In [7]:
import numpy as np
from numpy.linalg import inv

def lm_fs_fast(X_all, y, base_vars, use_const = True):
    n, p = X_all.shape
    if use_const:
        p -= 1
    X_base = X_all[:, base_vars]
    G_inv = inv(X_base.T @ X_base)
    b = X_base.T @ y
    total_var = np.sum((y - y.mean()) ** 2)

    r2s = {}
    base_set = set(base_vars)
    for j in range(p):
        if j in base_set:
            continue

        xj = X_all[:, j:j + 1]# (n,1)
        B = X_base.T @ xj
        C = xj.T @ X_base 
        D = xj.T @ xj

        # s = Schur complement
        G_inv_new = (C @ G_inv)
        S = D - (G_inv_new @ B)
        
        if S[0][0] <= 1e-10:
            r2s[j] = np.nan  # ill-conditioned
            continue

        G_inv_new_lower = -G_inv_new / S
        G_inv_updated = np.block([
            [G_inv + G_inv_new.T @ G_inv_new / S, G_inv_new_lower.T],
            [G_inv_new_lower, 1 / S]
        ])
        b_new = np.hstack((b, xj.T @ y))
        beta_new = G_inv_updated @ b_new    
        X_full = np.hstack([X_base, xj])
        y_pred = (X_full @ beta_new).flatten()
        ss_res = np.sum((y - y_pred) ** 2)
        r2 = 1 - ss_res / total_var
        r2s[j] = r2
    return r2s

def lm_el_fast(X, y, use_const = True):
    n, p = X.shape
    ids = np.arange(p)
    if use_const:
        p -= 1
    G_inv = inv(X.T @ X)
    b = X.T @ y
    total_var = np.sum((y - y.mean()) ** 2)
    r2s = np.zeros(p)
    
    for j in range(p):
        idx = ids != j

        # Extract M, alpha, v from full G_inv
        M = G_inv[np.ix_(idx, idx)] # (p-1)x(p-1)
        v = G_inv[idx, j].reshape(-1, 1)# (p-1)x1
        alpha = G_inv[j, j]# scalar
        # Schur-complement-based inverse of G_sub
        G_sub_inv = M - (v @ v.T) / alpha

        # b_sub for beta
        b_sub = b[idx].reshape(-1, 1) # (p-1)x1
        beta_sub = (G_sub_inv @ b_sub).flatten()  # (p-1)x1

        # Predict y using X_{-j}
        y_pred = X[:, idx] @ beta_sub
        ss_res = np.sum((y - y_pred) ** 2)
        r2s[j] = 1 - ss_res / total_var

    return r2s

In [8]:
import numpy as np
from numpy.linalg import inv
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

def lm_fs_fast2(X_all, y, base_vars, use_const = True, train_size = 0.8, random_state = 123):
    n, p = X_all.shape
    X_train, X_test, y_train, y_test = train_test_split(X_all, y, train_size = train_size, random_state = random_state)
    if use_const:
        p -= 1
        base_vars.append(-1)
    X_base = X_train[:, base_vars]
    G_inv = inv(X_base.T @ X_base)
    b = X_base.T @ y_train
    
    metric = {}
    base_set = set(base_vars)
    for j in range(p):
        if j in base_set:
            continue

        xj = X_train[:, j:j + 1]# (n,1)
        B = X_base.T @ xj
        C = xj.T @ X_base 
        D = xj.T @ xj

        # s = Schur complement
        G_inv_new = (C @ G_inv)
        S = D - (G_inv_new @ B)
        
        if S[0][0] <= 1e-10:
            metric[j] = np.inf
            continue

        G_inv_new_lower = -G_inv_new / S
        G_inv_updated = np.block([
            [G_inv + G_inv_new.T @ G_inv_new / S, G_inv_new_lower.T],
            [G_inv_new_lower, 1 / S]
        ])
        b_new = np.hstack((b, xj.T @ y_train))
        beta_new = (G_inv_updated @ b_new).flatten()
        X_full = np.hstack([X_test[:, base_vars], X_test[:, j:j + 1]])
        y_pred = X_full @ beta_new
        metric[j] = root_mean_squared_error(y_test, y_pred)
    return metric

def lm_el_fast2(X, y, use_const = True, train_size = 0.8, random_state = 123):
    n, p = X.shape
    ids = np.arange(p)
    if use_const:
        p -= 1
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = train_size, random_state = random_state)
    G_inv = inv(X_train.T @ X_train)
    b = X_train.T @ y_train
    metric = np.zeros(p)

    for j in range(p):
        idx = ids != j

        # Extract M, alpha, v from full G_inv
        M = G_inv[np.ix_(idx, idx)] # (p-1)x(p-1)
        v = G_inv[idx, j].reshape(-1, 1)# (p-1)x1
        alpha = G_inv[j, j]# scalar
        # Schur-complement-based inverse of G_sub
        G_sub_inv = M - (v @ v.T) / alpha

        # b_sub for beta
        b_sub = b[idx].reshape(-1, 1) # (p-1)x1
        beta_sub = (G_sub_inv @ b_sub).flatten()  # (p-1)x1

        # Predict y using X_{-j}
        y_pred = X_test[:, idx] @ beta_sub
        metric[j] = root_mean_squared_error(y_test, y_pred)
    return metric

In [9]:
def fs_fast(X, X_ff):
    return pd.Series(
        lm_fs_fast2(df_sel[X_ff + ['const']].values, df_sel[target], [X_ff.index(i) for i in X]).values(),
        index = [i for i in X_ff if i not in X]
    ).sort_values()

def be_fast(X, floated_list):
    if len(X) < 2: return pd.Series()
    return pd.Series(
        lm_el_fast2(df_sel[X + ['const']].values, df_sel[target]), index = X
    ).pipe(lambda x: x.loc[~x.index.isin(floated_list)]).sort_values()

def step_fs_fast(X_, X_selected, floated_list, rmse_list):
    while(True):
        s_rmse = fs_fast(X_selected, X_)
        if len(s_rmse) == 0:
            break
        if rmse_list[-1] > s_rmse.iloc[0]:
            rmse_list.append(s_rmse.iloc[0])
            X_selected.append(s_rmse.index[0])
            print("Selected: {}, RMSE: {}".format(X_selected[-1], rmse_list[-1]))
        else:
            break
        while(True):
            s_rmse = be_fast(X_selected, floated_list)
            if len(s_rmse) > 0 and rmse_list[-1] > s_rmse.iloc[0]:
                rmse_list.append(s_rmse.iloc[0])
                floated_list.add(s_rmse.index[0])
                X_selected = [i for i in X_selected if i != s_rmse.index[0]]
                print("Excluded: {}, RMSE: {}".format(s_rmse.index[0], rmse_list[-1]))
            else:
                break
    return X_selected, floated_list, rmse_list

In [10]:
from sklearn.preprocessing import StandardScaler
df_sel[X_sel] = StandardScaler().fit_transform(df_sel[X_sel])

In [11]:
result = sgml.cv(df_sel, ss, {'X_num': X_sel}, config, lr_adapter, result_proc = [sgml.lr_learning_result])
result['valid_scores']

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

[np.float64(0.0675481023860778)]

In [ ]:
X_selected = list()
rmse_list = [np.inf]
X_selected, floated_list, rmse_list = sc.cache_result(
    'fs_fast2_2', lambda : step_fs_fast(X_sel, X_selected, set(), rmse_list), rerun = 1
)

In [17]:
result = sgml.cv(df_sel, ss, {'X_num': X_selected}, config, lr_adapter, result_proc = [sgml.lr_learning_result])
result['valid_scores']

Fold:   0%|          | 0/1 [00:00<?, ?it/s]

[np.float64(0.06931439312353471)]

In [135]:
result['model_result'][0]['coef'].sort_values()

Weight_2                       -382.332152
Heart_Rate                     -161.056265
Heart_Rate_mul_Weight          -100.510612
Heart_Rate_sqrt_d               -87.574503
Weight_sqrt_d                   -72.084118
                                   ...    
Weight_3                         39.050229
Heart_Rate_r                     46.425879
Heart_Rate_div_Weight_sqrt_d     58.461733
Heart_Rate_2                    122.277537
Weight_sqrt                     356.378532
Length: 68, dtype: float64